### Network growth

1. Search "History" in section headings.
2. Find a list of years.
3. Use the first one as when the birth year of the idea.
4. If no "History" section, then look backwards in the graph to a page that does, and use that.